## Read CSV


In [5]:
import numpy as np
import pandas as pd
import heapq

In [6]:
df = pd.read_csv('us_routes_dist.csv')

In [7]:
df.head()

,Unnamed: 0,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,distance,from_to
0,172,2O,146,ADQ,3531,KLN,7162,NaN,0,BNI,2.128521,ADQ_KLN_2O
1,177,2O,146,KLN,7162,KYK,7161,NaN,0,BNI,0.603091,KLN_KYK_2O
2,260,3E,10739,BRL,5726,ORD,3830,NaN,0,CNC,1.153044,BRL_ORD_3E
3,261,3E,10739,BRL,5726,STL,3678,NaN,0,CNC,1.609324,BRL_STL_3E
4,262,3E,10739,DEC,4042,ORD,3830,NaN,0,CNC,11.138693,DEC_ORD_3E


## Create a graph

In [54]:
# Get uniques source-dest pairs from df
graph = df[['source', 'dest', 'distance']]

In [55]:
# Drop from graph where distance is NaN
graph = graph.dropna()
graph

,source,dest,distance
0,ADQ,KLN,2.128521
1,KLN,KYK,0.603091
2,BRL,ORD,1.153044
3,BRL,STL,1.609324
4,DEC,ORD,11.138693
...,...,...,...
2189,TUS,ATL,4.136367
2190,TYS,ATL,34.461664
2191,VLD,ATL,10.263855
2192,VPS,ATL,6.441787


In [56]:
graph.loc[graph['source'] == 'ADQ']

,source,dest,distance
0,ADQ,KLN,2.128521
99,ADQ,ANC,1.587634
280,ADQ,AKK,16.320888


In [57]:
graph.loc[(graph['source'] == 'ORD') & (graph['dest'] == 'ATL')]
print(graph.loc[(graph['source'] == 'ORD')])

print(graph.loc[graph['source'] == 'BRL'])

graph.loc[(graph['source'] == 'ORD')&((graph['dest'] == 'STL'))]

     source dest   distance
7       ORD  BRL   0.546768
8       ORD  DEC   0.305825
1390    ORD  ABQ   9.037408
1391    ORD  ALO   9.037408
1392    ORD  ART   0.842480
...     ...  ...        ...
1485    ORD  TVC  11.140467
1486    ORD  TYS  32.384502
1487    ORD  XNA  36.401936
1908    ORD  MSY   4.778303
2152    ORD  ATL  40.090622

[102 rows x 3 columns]
  source dest  distance
2    BRL  ORD  1.153044
3    BRL  STL  1.609324


,source,dest,distance
1478,ORD,STL,17.124137


In [58]:
graph_dict = {}

for row in df.itertuples():
    source = row.source
    dest = row.dest
    distance = row.distance
    
    if pd.isna(distance):
        continue

    if source not in graph_dict:
        graph_dict[source] = []
    
    graph_dict[source].append((dest, distance))

In [65]:
graph_dict

{'ADQ': [('KLN', 2.128521013470194),
  ('ANC', 1.5876342418321463),
  ('AKK', 16.32088811385432)],
 'KLN': [('KYK', 0.6030914575036689)],
 'BRL': [('ORD', 1.153043939982465), ('STL', 1.6093243966706878)],
 'DEC': [('ORD', 11.13869269275995), ('STL', 0.6431385370968897)],
 'JBR': [('STL', 1.3701737432323084)],
 'ORD': [('BRL', 0.546767573006186),
  ('DEC', 0.3058249659506772),
  ('ABQ', 9.037407550588908),
  ('ALO', 9.037407550588908),
  ('ART', 0.8424798669239778),
  ('ATL', 1.550020033026388),
  ('AUS', 1.2986829487489724),
  ('AZO', 1.3753358530752666),
  ('BDL', 1.3753358530752666),
  ('BMI', 1.221712486613395),
  ('BNA', 0.394881157749491),
  ('BOS', 0.5331243710229252),
  ('BUF', 0.8424798669239778),
  ('BWI', 0.7128993776051306),
  ('CHA', 1.550020033026388),
  ('CHO', 0.4285467870648635),
  ('CID', 1.221712486613395),
  ('CLE', 0.5331243710229252),
  ('CLT', 0.7128993776051306),
  ('CMH', 1.177191196754432),
  ('CMI', 8.762743788219026),
  ('COU', 9.037407550588908),
  ('CVG', 1

## Depth First Search


In [109]:
def dfs(graph, start, end, visited=None, path=None):
    if visited is None:
        visited = []
    if path is None:
        path = [start]

    visited.append(start)

    # Se o vértice de destino foi alcançado
    if start == end:
        return path

    # Pegando os vértices adjacentes
    neighbours = [neighbour for neighbour, dist in graph.get(start, [])]
    for neighbour in neighbours:
        if neighbour not in visited:
            new_path = dfs(graph, neighbour, end, visited, path + [neighbour])
            if new_path:  # Se um caminho válido foi encontrado, retorne-o
                return new_path

    return None  # Se nenhum caminho foi encontrado

O código acima gera uma lista , descrevendo todos os nós intermediários do caminho entre A e B. 

In [110]:
def calcula_distancia(start, path, distance):
    
    for i in path:

        distance += graph[(graph['source'] == start) & (graph['dest'] == i)]['distance'].iloc[0]
        start = i
    
    return distance

In [111]:
start = "ADQ"
end = "KLN"
dfs_result_dict = dfs(graph_dict, start, end, [], [])
dfs_result_dict

['KLN']

In [112]:
distance = calcula_distancia(start, dfs_result_dict, 0)
distance

2.128521013470194